In [1]:
pip list

Package                       Version
----------------------------- --------------------
alabaster                     0.7.12
anaconda-client               1.11.0
anaconda-navigator            2.3.1
anaconda-project              0.11.1
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.2
astroid                       2.11.7
astropy                       5.1
atomicwrites                  1.4.0
attrs                         21.4.0
Automat                       20.2.0
autopep8                      1.6.0
Babel                         2.9.1
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
backports.tempfile            1.0
backports.weakref             1.0.post1
bcrypt                        3.2.0
beautifulsoup4                4.11.1
binaryornot                   0.4.4
bitarray                      2.5.1
bkcharts                      0.2
blac

In [4]:
import datetime as dt
import wget
import os
import pandas as pd
from zipfile import ZipFile

In [5]:
ano_corrente, mes_corrente, dia_corrente = [dt.date.today().year, dt.date.today().month, dt.date.today().day]
ano_corrente, mes_corrente, dia_corrente

(2022, 12, 12)

In [9]:
# baixa dados da binance conforme ticker e timeframe selecionados para a pasta ./Dados/
# timeframes disponiveis: 12h 15m 1d 1h 1m 1mo 1s 1w 2h 30m 3d 3m 4h 5m 6h 8h
# tickers disponiveis: https://data.binance.vision/?prefix=data/spot/monthly/klines/
ticker = "BTCUSDT"
timeframe = "1m"
def baixa_dados_binance(ticker, timeframe):
    url = "https://data.binance.vision/data/spot/monthly/klines/"
    if not os.path.exists(f"./Dados/Processados/{ticker}-{timeframe}.csv"):
        for ano in range(2017, ano_corrente+1):
            for mes in range(1,12+1):
                mes = str(mes).zfill(2)
                if not ((os.path.exists(f"./Dados/temp/{ticker}-{timeframe}-{ano}-{mes}.zip"))):
                    try:
                        wget.download(f"{url}{ticker}/{timeframe}/{ticker}-{timeframe}-{ano}-{mes}.zip"
                                      , out = f"./Dados/temp/")
                        pass
                    except:
                        print(f"Falha ao baixar {url}{ticker}/{timeframe}/{ticker}-{timeframe}-{ano}-{mes}.zip")
                else:
                    print(f"{ano}/{mes} já baixado")
    else:
        print("ticker/timeframe já processado")
baixa_dados_binance(ticker, timeframe)

ticker/timeframe já processado


In [5]:
# só extrai o arquivo dado como parâmetro para a pasta ./Dados/temp
def extrai_arquivo(arq):
    try:
        ZipFile(arq, 'r').extractall('./Dados/temp/')
    except:
        print(f"erro ao extrair {arq}")

In [12]:
# cria uma lista de arquivos para extrair
lista_arquivos = os.listdir("./Dados/temp/")
lista_arquivos = [x for x in lista_arquivos if x.startswith(f"{ticker}-{timeframe}")]
lista_arquivos[-5:]

[]

In [10]:
# cria um dataframe vazio pra colocar todos os dados dentro
nomes = ["Open time","Open","High","Low","Close","Volume","Close time","Quote asset volume"
                             ,"Number of trades","Taker buy base asset volume","Taker buy quote asset volume","Ignore"]
df = pd.DataFrame(columns = nomes)
df

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore


In [11]:
# concatena tudo em um CSV e deixa na pasta ./Dados/Processados/
for arq in lista_arquivos:
    extrai_arquivo(f"./Dados/temp/{arq}")
    df = pd.concat([df, pd.read_csv(f'./Dados/temp/{arq[:-4]}.csv', sep=',',decimal='.'
                               , encoding='latin1', names=nomes, header=None)])
    print(arq)
    os.remove(f"./Dados/temp/{arq[:-4]}.csv")
df.drop("Ignore", inplace=True, axis=1)
df.set_index("Open time", inplace=True)
df.to_csv(f"./Dados/Processados/{ticker}-{timeframe}.csv")

NameError: name 'lista_arquivos' is not defined

In [9]:
# deleta tudo que é temporario
for arq in lista_arquivos:
    os.remove(f"./Dados/temp/{arq}")